## Generating Train Set

Creating Atom Models with Different Crystal Structures.

In [ ]:
from wizard.atoms import SymbolInfo, Morph
import numpy as np

train_set = []
SymbolInfos = [
   SymbolInfo('MoTaVW', 'bcc', 3.32),
]

Generating and Saving Deformed Atom Models with Random Strain and Displacement.

In [ ]:
from wizard.frames import MultiMol

frames = []
for SymbolInfo in SymbolInfos:
    atoms = SymbolInfo.create_bulk_atoms((2, 2, 2))
    frames.append(atoms)

deform_scale = np.arange(0.95, 1.06, 0.05)
strain_ratio = 0.04
max_displacement = 0.4

init_1 = MultiMol(frames).deform(deform_scale)
init_2 = MultiMol(init_1).random_strain(strain_ratio)
init_3 = MultiMol(init_1).random_displacement(max_displacement)

init = init_1 + init_2 + init_3

Generating Atom Models with Various Types of Defects

In [ ]:
from ase import Atoms

dimers = []
distances = [1.1 + i * 0.1 for i in range(10)]
for symbol_info in SymbolInfos:
    symbol1 = symbol_info.symbols[0]
    symbol2 = symbol_info.symbols[-1]
    for distance in distances:
        dimer = Atoms(symbols= [symbol1, symbol2], positions=[(0, 0, 0), (0, 0, distance)], pbc = [True, True, True] ,cell=(20, 30, 40))
        dimers.append(dimer)

In [ ]:
mono_vacancy = []
for symbol_info in SymbolInfos:
    atoms = symbol_info.create_bulk_atoms((3, 4, 5))
    Morph(atoms).create_vacancy()
    mono_vacancy.append(atoms)

In [ ]:
nths = [1,2,3,4,5]
di_vacancies = []
for symbol_info in SymbolInfos:
    for nth in nths:
        atoms = symbol_info.create_bulk_atoms((3, 4, 5)) 
        Morph(atoms).create_divacancies(nth)
        di_vacancies.append(atoms)

In [ ]:
vacancies = []
for symbol_info in SymbolInfos:
    for n in range(3, 10, 3):
        atoms = symbol_info.create_bulk_atoms((3, 4, 5)) 
        Morph(atoms).create_vacancies(n)
        vacancies.append(atoms)

In [ ]:
sia = []
vectors = [(1,1,1),(1,0,0),(1,1,0)]
for symbol_info in SymbolInfos:
    for vector in vectors:
        atoms = symbol_info.create_bulk_atoms((3,4,5)) 
        Morph(atoms).create_self_interstitial_atom(vector)
        sia.append(atoms)

In [ ]:
vectors = [(1,1,1),(1,0,0),(1,1,0)]
nths = [1,2,3,4,5]
di_sias = []
for symbol_info in SymbolInfos:
    for vector1 in vectors:
        for vector2 in vectors:
            for nth in nths:
                atoms = symbol_info.create_bulk_atoms((3,4,5))
                Morph(atoms).create_di_self_interstitial_atoms(vector1=vector1, vector2=vector2, nth=nth)
                di_sias.append(atoms)

In [ ]:
fks = []
for symbol_info in SymbolInfos:
    atoms = symbol_info.create_bulk_atoms((3,4,5))
    Morph(atoms).create_fks(10)
    fks.append(atoms)

In [ ]:
from ase.build import surface

millers = [(1,1,0),(0,0,1),(1,1,1),(1,1,2)]
surf = []
for symbol_info in SymbolInfos:
    for miller in millers:
        atoms = symbol_info.create_bulk_atoms()
        slab = surface(atoms, miller, layers = 10, vacuum=10) *(2, 2, 1)
        surf.append(slab)

Exploring Structures Using GPUMD

In [ ]:
from wizard.io import read_xyz
from wizard.frames import MultiMol

Temp_Struts = []
temperatures = [50, 300, 800, 1300, 1700, 2300, 3000, 4000, 5000]
for symbol_info in SymbolInfos:
    for temperature in temperatures:
        atoms = symbol_info.create_bulk_atoms((3,3,3))
        dirname = f'{symbol_info.formula}/{symbol_info.structure}/{temperature}K/relax'
        run_in=['potential ../../../../nep.txt',
                f'velocity {temperature}',   
                'time_step 1',
                f'ensemble npt_mttk temp {temperature} {temperature} iso 0 0',
                'dump_thermo 10000',
                'dump_exyz 200000',
                'dump_restart 1000000',
                'run 1000000']
        Morph(atoms).gpumd(dirname=dirname, run_in=run_in)
        Temp_Struts += read_xyz(dirname + '/dump.xyz')

deform_scale = np.arange(0.95, 1.06, 0.05)
strain_ratio = 0.05
Temp_Struts_Deform = MultiMol(Temp_Struts).deform(deform_scale)
Temp_Struts_Strain = MultiMol(Temp_Struts).random_strain(strain_ratio)

Temp_Struts = Temp_Struts_Deform + Temp_Struts_Strain
MultiMol(Temp_Struts).dump('Temp_Struts.xyz')

In [ ]:
from wizard.io import read_xyz, read_restart
from wizard.frames import MultiMol

temperature = 300 #K
strain_rate = 2e8
dt = 1e-15 #fs
utc_tensile = []
for symbol_info in SymbolInfos:
    atoms = read_restart(f'{symbol_info.symbol}/{symbol_info.structure}/{temperature}K/relax/restart.xyz')
    dirname = f'{symbol_info.symbol}/{symbol_info.structure}/{temperature}K/utc_tensile'
    length = atoms.cell[2, 2]
    strain = strain_rate * dt * length
    run_in = ['potential ../../../../nep.txt', 
              'velocity 300', 
              'time_step 1',
              f'ensemble npt_scr {temperature} {temperature} 100 0 0 0 100 100 100 1000',
              f'deform {strain} 0 0 1', 
              'dump_thermo 1000', 
              'dump_exyz 200000', 
              'dump_restart 10000',
              'run 2000000']
    Morph(atoms).gpumd(dirname=dirname, run_in=run_in)
    utc_tensile += read_xyz(dirname + '/dump.xyz') 

MultiMol(utc_tensile).dump('utc_tensile.xyz')

In [ ]:
from wizard.io import read_xyz
from wizard.frames import MultiMol

Temp_Struts = []
temperatures = [50, 300, 800, 1300, 1700, 2300, 3000, 4000, 5000]
for i, atoms in enumerate(frames):
    for temperature in temperatures:
        dirname = f'defects/{i}/{temperature}K/relax'
        run_in=['potential ../../../../nep.txt',
                f'velocity {temperature}',   
                'time_step 1',
                f'ensemble npt_mttk temp {temperature} {temperature} iso 0 0',
                'dump_thermo 10000',
                'dump_exyz 200000',
                'dump_restart 1000000',
                'run 1000000']
        Morph(atoms).gpumd(dirname=dirname, run_in=run_in)
        Temp_Struts += read_xyz(dirname + '/dump.xyz')

MultiMol(Temp_Struts).dump('Defect_Temp_Struts.xyz')